In [2]:
import sys
!{sys.executable} -m pip install -U turicreate matplotlib opencv-python
import turicreate as tc
import numpy as np
import os
import glob

In [43]:
from PIL import Image
from matplotlib import pyplot as plt
import cv2

images = []
image_labels = []
files = glob.glob ("data/images/*")
for image_file in files:
#     img = Image.open(image_file)
#     data = np.array( img, dtype='uint8' )
#     image = cv2.imread(image_file)
    im = Image.open(image_file)
    im = np.array(im)
    images.append(im)
#     images.append(image)
#     image_labels.append("arrow")

images = np.array(images)
np.save('data/bitmaps/arrow.npy', images)

# visually testing our output
img_array = np.load('data/bitmaps/arrow.npy')
# plt.imshow(img_array) 

In [56]:
# mostly taken from: https://github.com/apple/turicreate/blob/master/userguide/drawing_classifier/data-preparation.md

random_state = np.random.RandomState(100)

# Change if applicable
data_dir = 'data'
bitmaps_dir = os.path.join(data_dir, 'bitmaps')
sframes_dir = os.path.join(data_dir, 'sframes')
npy_ext = '.npy'
num_examples_per_class = 100
classes = ["arrow", "circle"]
should_reshape = {"circle": True, "arrow": False}
num_classes = len(classes)

def build_bitmap_sframe():
    bitmaps_list, labels_list = [], []
    for class_name in classes:
        class_data = np.load(os.path.join(bitmaps_dir, class_name + npy_ext))
        random_state.shuffle(class_data)
        class_data_selected = class_data[:num_examples_per_class]
        if should_reshape[class_name]:
            class_data_selected = class_data_selected.reshape(
                class_data_selected.shape[0], 28, 28, 1
            )
        for np_pixel_data in class_data_selected:
            FORMAT_RAW = 2
            bitmap = tc.Image(_image_data = np_pixel_data.tobytes(),
                              _width = np_pixel_data.shape[1],
                              _height = np_pixel_data.shape[0],
#                               _channels = np_pixel_data.shape[2],
                              _channels = 1,
                              _format_enum = FORMAT_RAW,
                              _image_data_size = np_pixel_data.size)
            bitmaps_list.append(bitmap)
            labels_list.append(class_name)

    sf = tc.SFrame({"drawing": bitmaps_list, "label": labels_list})
    sf.save(os.path.join(sframes_dir, "bitmap_data.sframe"))
    return sf 

sf = build_bitmap_sframe()

In [51]:
sf.explore()

/Users/marekfort/opt/miniconda3/envs/automata-editor/lib/python3.8/site-packages/turicreate/visualization/_plot.py:461: UserWarning: Displaying only the first 100 rows.
  warnings.warn("Displaying only the first {} rows.".format(maximum_rows))


,drawing,label
0,,arrow
1,,arrow
2,,arrow
3,,arrow
4,,arrow
5,,arrow
6,,arrow
7,,arrow
8,,arrow
9,,arrow


In [57]:
SFRAME_PATH = "data/sframes/bitmap_data.sframe"
data =  tc.SFrame(SFRAME_PATH)

In [53]:
data.explore()

/Users/marekfort/opt/miniconda3/envs/automata-editor/lib/python3.8/site-packages/turicreate/visualization/_plot.py:461: UserWarning: Displaying only the first 100 rows.
  warnings.warn("Displaying only the first {} rows.".format(maximum_rows))


,drawing,label
0,,arrow
1,,arrow
2,,arrow
3,,arrow
4,,arrow
5,,arrow
6,,arrow
7,,arrow
8,,arrow
9,,arrow


In [58]:
train_data, test_data = data.random_split(0.7)

In [59]:
model = tc.drawing_classifier.create(train_data, 'label', max_iterations=20)

Automatically generating validation set from 10% of the data.

Using CPU to create model.

+--------------+----------------+---------------------+--------------+

| Iteration    | Train Accuracy | Validation Accuracy | Elapsed Time |

+--------------+----------------+---------------------+--------------+

...

| 5            | 0.980392       | 1                   | 3.37s        |

| 6            | 0.980392       | 1                   | 3.79s        |

| 11           | 0.980392       | 1                   | 6.26s        |

| 20           | 0.990196       | 1                   | 10.51s       |

+--------------+----------------+---------------------+--------------+

In [101]:
model

Class                                    : DrawingClassifier

Schema
------
Number of classes                        : 3
Feature column                           : drawing
Target column                            : label

Training summary
----------------
Training Iterations                      : 20
Training Accuracy                        : 1.0
Validation Accuracy                      : 0.8824
Training Time                            : 13.7598
Number of Examples                       : 153

In [69]:
predictions = model.predict(test_data)

In [60]:
metrics = model.evaluate(test_data)
metrics.explore()

Resizing image data... Done.


In [61]:
model.save("automata-editor.model")

In [62]:
model.export_coreml("AutomataClassifier.mlmodel")